<a href="https://colab.research.google.com/github/pure-water/utility_ml/blob/master/student_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 This note book is based on FIR2003 dataset and a base model (Resnet and Mobilenet) for class student detection

In [2]:
from tensorflow import keras
# from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [3]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from random import shuffle

In [4]:
! pip install kaggle
! mkdir ~/.kaggle
# Copy Twice, whatever 
! cp /kaggle.json ~/.kaggle/
! cp /content/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! ls ~/


cp: cannot stat '/kaggle.json': No such file or directory
cp: cannot stat '/content/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [5]:
!kaggle datasets download  msambare/fer2013

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [6]:
!unzip fer2013

unzip:  cannot find or open fer2013, fer2013.zip or fer2013.ZIP.


In [8]:
classes_num = 7
classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

#YaoGang Colab Setup
Train_DIR = "/content/train"
Test_DIR = "/content/test"


#leagacy setup
##Train_DIR = "../input/fer2013/train"
##Test_DIR = "/content/test"

# one-hot:
def label_img(category):
    arr = np.zeros((classes_num,), dtype = int)
    arr[classes.index(category), ] = 1
    return arr

# Datasets:

img_size = 224
train = []
test = []
def create_train_set(img_size):
    for category in classes:
        imgs = 0
        category_path = os.path.join(Train_DIR, category)
        label = label_img(category)
        for img in os.listdir(category_path):
            imgs = imgs + 1
            if imgs == 4000:
                break
            img_array = cv2.resize(cv2.imread(os.path.join(category_path, img)), (img_size, img_size))
            train.append([np.array(img_array), np.array(label)])

create_train_set(img_size)
train = np.array(train)
test  = np.array(test)
shuffle(train)

print("Train shape",train.shape)
print("Test shape",test.shape)

FileNotFoundError: ignored

In [ ]:
def plot_images(img_dir, top=10):
    all_img_dirs = os.listdir(img_dir)
    img_files = [os.path.join(img_dir, file) for file in all_img_dirs][:5]
  
    plt.figure(figsize=(10, 10))
  
    for idx, img_path in enumerate(img_files):
        plt.subplot(5, 5, idx+1)
        img = plt.imread(img_path)
        plt.tight_layout()         
        plt.imshow(img, cmap='gray') 

In [ ]:
plot_images('train'+'/angry')


In [ ]:
plot_images('train'+'/happy')

In [ ]:
plot_images('train'+'/neutral')

In [ ]:
# CREATING VALIDATION SET FROM A SUBSET OF TRAINING SET:

# valTOtrain_split_ratio = 0.2  # (val / total)
# num = int(((1 - valTOtrain_split_ratio)*train.shape[0]))
# train_set = train[:num]
# val_set = train[num:]

In [ ]:
import gc
split = train.shape[0]*0.8
print("split threshold", split)
X_train = np.array([i[0] for i in train[:split]]).reshape(-1, img_size, img_size, 3)
y_train = np.array([i[1] for i in train[:split]])

print(X_train.shape)
print(y_train.shape)

X_val = np.array([i[0] for i in train[split:]]).reshape(-1, img_size, img_size, 3)
y_val = np.array([i[1] for i in train[split:]])

print(X_val.shape)
print(y_val.shape)

del train
gc.collect()

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow import keras 

In [ ]:
# We shall not build a new model from scratch, rather we will use transfer learning:

img_size = 224
classes_num = 7
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow.keras.applications

resnet_model = ResNet50(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')
mobilenet_model = tf.keras.applications.MobileNetV2(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')

In [ ]:
for layers in mobilenet_model.layers: 
  layers.trainable = False

In [ ]:
# Model1 using the mobile_net as the base model

model2 = Sequential()
model2.add(mobilenet_model)

model2.add(Flatten())
model2.add(BatchNormalization())
model2.add(Dense(256, activation='relu'))
model2.add(Dense(classes_num, activation='softmax'))

model2.summary()

In [ ]:
checkpoint_path= './'
callback=tf.keras.callbacks.ModelCheckpoint(
     checkpoint_path, verbose=1, save_weights_only=True,
    period=5
)

In [ ]:
model2.compile(optimizer=Adam(learning_rate=0.0003),loss='categorical_crossentropy',metrics=['accuracy'])
history2 = model2.fit(X_train, y_train, batch_size = 64, epochs=18, validation_data=(X_val, y_val))

In [ ]:
from google.colab import files
model_file = "model_mobilenet_input_224.h5"
model2.save_weights(model_file)
files.download(model_file)

In [ ]:
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model2.save("model_mobilenet_input_224")

In [ ]:
import shutil
shutil.make_archive("model_mobilenet", 'zip', "./model_mobilenet_input_224")

In [ ]:
# Model1 using the resnet_model as the base model and fine tuning it

for layer in resnet_model.layers:
   layer.trainable = False
for layer in resnet_model.layers[-75:]:
   layer.trainable = True

# **IMPORTANT**
# let's visualize layer names and layer indices to see how many layers we should freeze:
# for i, layer in enumerate(resnet_model.layers):
#    print(i, layer.name)

model1 = Sequential()
model1.add(resnet_model)

model1.add(Flatten())
model1.add(BatchNormalization())

model1.add(Dense(classes_num, activation='softmax'))

model1.summary()

In [ ]:
model1.compile(optimizer=Adam(learning_rate=0.0003),loss='categorical_crossentropy',metrics=['accuracy'])
history1 = model1.fit(X_train, y_train, batch_size = 64, epochs=25, validation_data=(X_val, y_val))

In [ ]:
model1.save_weights("model_resnet_FineTuned_25ep.h5")

In [ ]:
# import shutil
# shutil.make_archive("model_resnet_FineTuned", 'zip', "./model_resntnet_FineTuned")

In [ ]:
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()